<a href="https://colab.research.google.com/github/nicolly-sak/HD-emg-analysis/blob/main/PDPD4_Correla%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import pickle 
from matplotlib import pylab
from google.colab import widgets
import scipy.signal
!pip install hickle
import hickle
!pip install --upgrade numpy
import h5py
from numba import jit
import numba
import time
!pip install --upgrade tbb

In [ ]:
dir_list = hickle.load('/content/drive/MyDrive/PDPD/Dicionário/dir_list.hkl')

In [ ]:
dir_list = hickle.load('/content/drive/MyDrive/PDPD/Dicionário/dir_list.hkl')
dados01 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados01.h5", "r+")
dados02 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados02.h5", "r+")
dados03 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados03.h5", "r+")
dados04 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados04.h5", "r+")
dados05 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados05.h5", "r+")
dados06 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados06.h5", "r+")
dados07 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados07.h5", "r+")
dados08 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados08.h5", "r+")
dados09 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados09.h5", "r+")
dados10 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados10.h5", "r+")
dados11 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados11.h5", "r+")
dados12 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados12.h5", "r+")
dados13 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados13.h5", "r+")
dados14 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados14.h5", "r+")
dados15 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados15.h5", "r+")
dados16 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados16.h5", "r+")
dados17 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados17.h5", "r+")
dados18 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados18.h5", "r+")
dados19 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados19.h5", "r+")
dados20 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados20.h5", "r+")
dados21 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados21.h5", "r+")
dados22 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados22.h5", "r+")
dados23 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados23.h5", "r+")
dados24 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados24.h5", "r+")
dados25 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados25.h5", "r+")

In [ ]:
dados24 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados24.h5", "r+")
dados25 = h5py.File("/content/drive/MyDrive/PDPD/Dicionário/dados25.h5", "r+")

In [ ]:
tabelas_correlacao_dict = hickle.load('/content/drive/MyDrive/PDPD/Dicionário/tabelas_correlacao_dict.hkl')

In [ ]:
tabelas_correlacao_dict[24]

KeyError: ignored

In [ ]:
def make_combinations_1d(x,y):
    len_x = x.shape[0]
    len_y = y.shape[0]

    if x.dtype == np.float64 or x.dtype == np.float32 or x.dtype == np.float16:
      X = np.tile(x,(len(y),1)).astype(np.float64)
      Y = np.tile(y, (1,len_x)).reshape((y.shape[0]*len_x,y.shape[1])).astype(np.float64)
    else:
      X = np.tile(x,(len(y),1))
      Y = np.tile(y, (1,len_x)).reshape((y.shape[0]*len_x,y.shape[1]))

    return X, Y

In [ ]:
@jit("f8[:,:](f8[:,:], f8[:,:])", nopython=True, parallel=True, fastmath=True)
def compute_correlation(x,y):
  c = np.zeros((x.shape[0], y.shape[1]-x.shape[1]))
  N = x.shape[1]
  for i in range(c.shape[1]-1):
    c[:,i] = np.sum(x*y[:,i:i+N], axis=1)
  c = c/N 
  return c


def crosscorr2(x,y, lags_max=1000,  alpha=0.05):
    '''
     Computes the normalized cross-correlation (formula 5.3 from Billings, SA (2013)) between the signals x and y.
     
     written by: Renato Naville Watanabe 
     
     [phi, lags, CB] = crosscorr(x,y, alpha)
     	
     
     Inputs:
       
       x and y: vector of np.float64s, column-vectors with the signals to compute the cross-correlation.
     
       alpha: np.float64, significance value of the confidence boundaries. Usually is used alpha = 0.05.
     
     
     Outputs:
     
       phi: vector of np.float64s, the normalized crosscorrelation.
     
       lags: vector of integers, vector with the corresponding lags of the phi vector.
     
       CB: vector of 2 np.float64 elements, confidence boundaries to consider that the cross-correlation at a given value is zero.
    '''
    
    
    x = x - np.mean(x, axis=1, keepdims=True)
    
    # stdx = np.std(x, axis=1, keepdims=True)
    # stdy = np.std(y, axis=1, keepdims=True)
    y = y - np.mean(y, axis=1, keepdims=True)

    y = np.hstack((np.zeros((y.shape[0], lags_max//2), dtype=y.dtype), y, np.zeros((y.shape[0], lags_max//2), dtype=y.dtype)))
   
    c = compute_correlation(x,y)
    # c = c/(stdx*stdy+1e-6)
    
    N = c.shape[1]
    
    # # CB = np.array([-np.max(np.abs(phir)), np.max(np.abs(phir))])
    lags = np.arange(N)
    lags = lags - N//2
    return c, lags

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
#tabelas_correlacao_dict = dict()

fs_glove = 256
dt = 1/fs_glove


start = time.time()


def tabela_correlacao(n, dados) :

  colunas = ['subj','movement','channels_emg','channels_glove','correlacao','session']
  tabela_correlacaox = pd.DataFrame(columns=colunas)

  for d in range(len(dados)):
    if dir_list[0][d].find('calibration_values') < 0:    
      emg_sd_decimated = scipy.signal.decimate(pd.DataFrame(np.abs(dados['/'+str(d)+'/emg'][()])).rolling(window= 205, min_periods= 0,
                                                                                    center= True, axis=1).std().values, 
                                              8, n=None , ftype='iir', axis=1, zero_phase=True)
      
      glove_dict = dados['/'+str(d)+'/glove'][()]
    
      emg_sd_decimated,glove_dict = make_combinations_1d(emg_sd_decimated,glove_dict)
      emg_channels, glove_channels = make_combinations_1d(dados['/'+str(d)+'/channels_emg'][()].reshape(-1,1),dados['/'+str(d)+'/channels_glove'][()].reshape(-1,1))
      
      corr, lags = crosscorr2(glove_dict, emg_sd_decimated, lags_max=256)
      
      
      max_corr = lags[np.argmax(np.abs(corr), axis=1)]*dt
      
      new_tabela_correlacao= pd.DataFrame(columns=colunas)
      new_tabela_correlacao['channels_emg'] = emg_channels.squeeze()
      new_tabela_correlacao['channels_glove'] = glove_channels.squeeze()
      new_tabela_correlacao['subj'] = dados['/'+str(d)+'/subject'][()][0]
      new_tabela_correlacao['movement'] = dados['/'+str(d)+'/movement'][()][0]
      new_tabela_correlacao['session'] = dados['/'+str(d)+'/session'][()][0][0]
      new_tabela_correlacao['speed'] = dados['/'+str(d)+'/speed'][()][0]
      new_tabela_correlacao['correlacao'] = max_corr

      tabela_correlacaox = tabela_correlacaox.append(new_tabela_correlacao)
  end = time.time()
  print("Elapsed = %s" % (end - start))

  tabelas_correlacao_dict[n] = tabela_correlacaox
    

In [ ]:
tabela_correlacao(2, dados02)
tabela_correlacao(3, dados03)

Elapsed = 3712.2408394813538
Elapsed = 7452.175440788269


In [ ]:
tabela_correlacao(4, dados04)
tabela_correlacao(5, dados05)

Elapsed = 13487.355057239532
Elapsed = 18545.76593708992


In [ ]:
tabela_correlacao(6, dados06)
tabela_correlacao(7, dados07)

Elapsed = 3644.266088962555
Elapsed = 7788.981626987457


In [ ]:
tabela_correlacao(8, dados08)
tabela_correlacao(9, dados09)

Elapsed = 3055.708456516266
Elapsed = 6573.24555850029


In [ ]:
tabela_correlacao(10, dados10)
tabela_correlacao(11, dados11)

Elapsed = 2588.736430644989
Elapsed = 5554.144231557846


In [ ]:
tabela_correlacao(12, dados12)
tabela_correlacao(13, dados13)

Elapsed = 9645.682575941086
Elapsed = 13701.768687963486


In [ ]:
tabela_correlacao(14, dados14)
tabela_correlacao(15, dados15)

Elapsed = 2587.1750576496124
Elapsed = 6040.453419208527


In [ ]:
tabela_correlacao(16, dados16)
tabela_correlacao(17, dados17)

Elapsed = 3785.253235578537
Elapsed = 7827.74821972847


In [ ]:
tabela_correlacao(18, dados18)
tabela_correlacao(19, dados19)

Elapsed = 3608.9057829380035
Elapsed = 8081.772054195404


In [ ]:
tabela_correlacao(20, dados20)
tabela_correlacao(21, dados21)

Elapsed = 3474.7226202487946
Elapsed = 7361.485599040985


In [ ]:
tabela_correlacao(22, dados22)
tabela_correlacao(23, dados23)

Elapsed = 16984.90095257759
Elapsed = 22279.102703094482


In [ ]:
tabela_correlacao(24, dados24)
tabela_correlacao(25, dados25)

Elapsed = 2780.0059299468994
Elapsed = 5836.647820234299


In [ ]:
hickle.dump(tabelas_correlacao_dict, '/content/drive/MyDrive/PDPD/Dicionário/tabelas_correlacao_dict.hkl', mode='w')

/usr/local/lib/python3.7/dist-packages/hickle/loaders/load_builtins.py:110: SerializedWarning: 'DataFrame' type not understood, data has been serialized
  % (py_obj.__class__.__name__, reason_str), SerializedWarning)


In [ ]:
 tabela_correlacao(1, dados01)
tabela_correlacao(2, dados02)
tabela_correlacao(3, dados03)
tabela_correlacao(4, dados04)
tabela_correlacao(5, dados05)
tabela_correlacao(6, dados06)
tabela_correlacao(7, dados07)
tabela_correlacao(8, dados08)
tabela_correlacao(9, dados09)
tabela_correlacao(10, dados10)
tabela_correlacao(11, dados11)
tabela_correlacao(12, dados12)
tabela_correlacao(13, dados13)
tabela_correlacao(14, dados14)
tabela_correlacao(15, dados15)
tabela_correlacao(16, dados16)
tabela_correlacao(17, dados17)
tabela_correlacao(18, dados18)
tabela_correlacao(19, dados19)
tabela_correlacao(20, dados20)
tabela_correlacao(21, dados21)
tabela_correlacao(22, dados22)
tabela_correlacao(23, dados23)
tabela_correlacao(24, dados24)
tabela_correlacao(25, dados25)

Elapsed = 5099.683191776276
Elapsed = 10252.30755162239
Elapsed = 15687.039838552475


/usr/local/lib/python3.7/dist-packages/hickle/loaders/load_builtins.py:110: SerializedWarning: 'DataFrame' type not understood, data has been serialized
  % (py_obj.__class__.__name__, reason_str), SerializedWarning)


Elapsed = 21142.966209173203


Hickle

In [ ]:
hickle.dump(tabelas_correlacao_dict, '/content/drive/MyDrive/PDPD/Dicionário/tabelas_correlacao_dict.hkl', mode='w')

NameError: ignored

h5

In [ ]:
!pip install git+https://github.com/rnwatanabe/silx.git
import silx.io.dictdump as dictdump

create_ds_args = {'compression': "gzip",
                   'shuffle': True,
                   'fletcher32': True}

dictdump.dicttoh5(tabela_correlacao, "/content/drive/MyDrive/PDPD/Dicionário/tabela_correlacao.h5", h5path="/",
          create_dataset_args=create_ds_args)

  Cloning https://github.com/rnwatanabe/silx.git to /tmp/pip-req-build-_yelyjhc
  Running command git clone -q https://github.com/rnwatanabe/silx.git /tmp/pip-req-build-_yelyjhc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.5 MB 8.0 MB/s 
  Created wheel for silx: filename=silx-0.16.0a0-cp37-cp37m-linux_x86_64.whl size=9877877 sha256=bd8a8d3d2917fa755207ef41602e452c341185dfd30f0510458de8d0f0267206
  Stored in directory: /tmp/pip-ephem-wheel-cache-6p2emgt6/wheels/7e/59/e5/8cd0bdbe60c0e9ea6bbed3f66436e136c181f4105c4512ced6
Successfully built silx


TypeError: ignored

---------------------------------------------------------------------------------------------------------------